In [ ]:
from PIL import Image, ImageDraw, ImageFont


In [ ]:
class Pattern:
    def __init__(self, file_path):
        self.pattern = {}
        self.read_pattern(file_path)

    def extract_stitch_info(self, stitch_info):
        stitch_count = ''
        for char in reversed(stitch_info):
            if char.isdigit():
                stitch_count = char + stitch_count
            else:
                break
        stitch_type = stitch_info[:len(stitch_info) - len(stitch_count)]
        return stitch_type, int(stitch_count) if stitch_count else 1

    def read_pattern(self, file_path):
        with open(file_path, 'r') as file:
            for i, line in enumerate(file, start=1):
                key = f'Row_{i}'
                stitches_info = line.strip().split(', ')
                stitches = []

                for stitch_info in stitches_info:
                    stitch_type, stitch_count = self.extract_stitch_info(stitch_info)
                    stitches.extend([stitch_type] * stitch_count)

                self.pattern[key] = stitches



In [ ]:
class Chart:
    def __init__(self, stitches_per_row, num_of_rows, chart_name, pattern, stitch_library):
        self.stitches_per_row = stitches_per_row
        self.num_of_rows = num_of_rows
        self.chart_name = chart_name
        self.pattern = pattern
        self.stitch_library = stitch_library
        self.initialize_chart()

    def initialize_chart(self):
        self.background = Image.new("RGB", ((self.stitches_per_row + 4) * stitch, (self.num_of_rows + 2) * stitch),
                                  (255, 255, 255))
        self.width, self.height = self.background.size
        self.x_filling = stitch + 0.93 * stitch * (self.stitches_per_row - 1)
        self.x_margin = round((self.width - self.x_filling) / 2)
        self.y_filling = stitch + 0.93 * stitch * (self.num_of_rows - 1)
        self.y_margin = round((self.height - self.y_filling) / 2)
        self.draw = ImageDraw.Draw(self.background)

    def draw_chart(self):
        for row_index, row_key in enumerate(self.pattern.pattern):
            row = self.pattern.pattern[row_key]
            double_stitch_count = 0

            for stitch_index, stitch_type in enumerate(row):
                stitch_image = self.stitch_library[stitch_type]
                if stitch_type in {'RC', 'LC', 'RT', 'LT', 'RT_purl', 'LT_purl', 'front_kk', 'front_kp',
                                   'front_pk', 'front_pp', 'back_kk', 'back_kp', 'back_pk', 'back_pp'}:
                    double_stitch_count += 1
                    x_position = self.x_margin + ((self.stitches_per_row - stitch_index - 1 - double_stitch_count)
                                                  * round(0.93 * stitch))
                else:
                    x_position = self.x_margin + (self.stitches_per_row - stitch_index - 1 - double_stitch_count) \
                                 * (round(0.93 * stitch))

                y_position = self.y_margin + (self.num_of_rows - row_index - 1) * round(0.93 * stitch)

                self.background.paste(stitch_image, (x_position, y_position))
                self.draw_arrows_and_numbers(row_index)

        for n_stitch in range(0, self.stitches_per_row - 2):
            self.draw.text((25 + self.x_margin + stitch + (self.stitches_per_row - 2 - n_stitch - 1)
                            * (int(0.93 * stitch)), self.y_margin - 75), f"{n_stitch + 1}",
                           fill=(0, 0, 0), font=ImageFont.truetype("times.ttf", 45))

    def draw_arrows_and_numbers(self, row_index):
        font_type = ImageFont.truetype("times.ttf", 60)

        if row_index % 2 == 0:
            l_text_position = (int(self.x_margin + self.x_filling) + 115,
                               20 + self.y_margin + (self.num_of_rows - row_index - 1) * round(0.93 * stitch))
            r_arrow_position = (int(self.x_margin + self.x_filling) + 30,
                                self.y_margin + int(stitch / 2) - int(left_arrow.size[1] / 2)
                                + (self.num_of_rows - row_index - 1) * round(0.93 * stitch))
            self.background.paste(left_arrow, r_arrow_position)
        elif row_index % 2 == 1:
            l_arrow_position = (self.x_margin - 110,
                                self.y_margin + int(stitch / 2) - int(left_arrow.size[1] / 2)
                                + (self.num_of_rows - row_index - 1) * round(0.93 * stitch))
            self.background.paste(right_arrow, l_arrow_position)

        self.draw.text(l_text_position, f"{row_index + 1}", fill=(0, 0, 0), font=font_type)

    def draw_report_lines(self, start_column, end_column):
        upper_l_arrow_position_1 = (self.x_margin - int(upper_arrow.size[0] / 2) - 4 + stitch
                                    + (self.stitches_per_row - start_column - 1) * round(0.93 * stitch),
                                    self.height - 100)
        upper_l_arrow_position_2 = (self.x_margin - int(upper_arrow.size[0] / 2) - 4 + stitch
                                    + (self.stitches_per_row - end_column - 1 - 1) * round(0.93 * stitch),
                                    self.height - 100)

        # Calculate the positions for the vertical lines
        line_start_position_1 = (self.x_margin - 3 + stitch
                                 + (self.st
